In [1]:
import poamlib
import pandas as pd
import json
from tqdm import tqdm
import poam_pb2
import base64

In [2]:
# Load input data from JSON file
with open("experiments.json", "r") as infile:
    input_data = json.load(infile)
df = pd.DataFrame(input_data).T
df["calculated"] = df.apply(lambda x: x["a"]* x["b"], axis=1)
df["cumulative"] = df["calculated"].cumsum()

In [3]:
def final_step_footprinting(df: pd.DataFrame, input_data: json) -> pd.DataFrame:
    df1 = pd.DataFrame(columns=["result", "proof_size", "seconds", "response"])
    method_payload = {}
    for i in tqdm(range(0, len(df.index))):
        method_payload[df.index[i]] = input_data[df.index[i]]
        with poamlib.experiment_timer() as elapsed:
            response, proof_size = poamlib.combined_request(json.dumps(method_payload))
            result = response["verificationValue"]
            time = elapsed()
            df1 = pd.concat([df1, pd.DataFrame({df.index[i]:{ "result": result, "proof_size": proof_size, "seconds": time, "response": response}}).T])
            #print(f"Experiment {i} took {time} seconds and yeilded a result of {result} with a proof size of {proof_size}")
    return pd.concat([df.copy(), df1], axis=1)

In [4]:
def activity_footprinting_unchained(df: pd.DataFrame, input_data: json)-> pd.DataFrame: 
    df1 = pd.DataFrame(columns=["result", "proof_size", "seconds", "response"])
    for i in tqdm(range(0, len(df.index))):
        method_payload = json.dumps({
            "a": df.loc[df.index[i]]["a"],
            "b": df.loc[df.index[i]]["b"],
            "operation":df.loc[df.index[i]]["operation"]
            })
        with poamlib.experiment_timer() as elapsed:
            response, proof_size = poamlib.proof_request(method_payload)
            result = response["verificationValue"]
            time = elapsed()
            df1 = pd.concat([df1, pd.DataFrame({df.index[i]:{ "result": result, "proof_size": proof_size, "seconds": time, "response": response}}).T])
            #print(f"Experiment {i} took {time} seconds and yeilded a result of {result} with a proof size of {proof_size}")
    return pd.concat([df.copy(), df1], axis=1)

In [5]:
def composition_footprinting_unchained(df_clean:pd.DataFrame, df: pd.DataFrame)->pd.DataFrame:
    df1 = pd.DataFrame(columns=["result", "proof_size", "seconds", "response"])
    proof_chain = []
    for i in tqdm(range(0, len(df.index))):
        input_data = df.loc[df.index[i]]["response"]
        proof_chain.append(poam_pb2.Proof(
                    image_id = input_data["proof"]["imageId"],
                    receipt = base64.b64decode(input_data["proof"]["receipt"].encode("utf-8"))
                ))
        with poamlib.experiment_timer() as elapsed:
            response, proof_size = poamlib.compose_request(proof_chain)
            result = response["verificationValue"]
            time = elapsed()
            df1 = pd.concat([df1, pd.DataFrame({df.index[i]:{ "result": result, "proof_size": proof_size, "seconds": time, "response": response}}).T])
            #print(f"Experiment {i} took {time} seconds and yeilded a result of {result} with a proof size of {proof_size}")
    return pd.concat([df_clean.copy(), df1], axis=1)

In [6]:
def activity_footprinting_chained(df: pd.DataFrame, input_data: json)-> pd.DataFrame:   
    df1 = pd.DataFrame(columns=["result", "proof_size", "seconds", "response"])
    previous = None
    chained = False
    for i in tqdm(range(0, len(df.index))):
        method_payload = json.dumps({
            "a": df.loc[df.index[i]]["a"],
            "b": df.loc[df.index[i]]["b"],
            "operation":df.loc[df.index[i]]["operation"]
            })
        if i > 0:
            previous = df1.loc[df.index[i-1]]["response"]
            chained = True
        with poamlib.experiment_timer() as elapsed:
            response, proof_size = poamlib.proof_request(method_payload, previous, chained)
            result = response["verificationValue"]
            time = elapsed()
            df1 = pd.concat([df1, pd.DataFrame({df.index[i]:{ "result": result, "proof_size": proof_size, "seconds": time, "response": response}}).T])
            #print(f"Experiment {i} took {time} seconds and yeilded a result of {result} with a proof size of {proof_size}")
    return pd.concat([df.copy(), df1], axis=1)

In [7]:
def proof_verification(df: pd.DataFrame)-> pd.DataFrame:
    df1 = pd.DataFrame(columns=["verified_value", "valid", "verification_seconds"])
    for i in tqdm(range(0, len(df.index))):
        response = df.loc[df.index[i]]["response"]
        proof = poam_pb2.Proof(
                image_id = response["proof"]["imageId"],
                receipt = base64.b64decode(response["proof"]["receipt"].encode("utf-8"))
            )
        with poamlib.experiment_timer() as elapsed:
            valid, result = poamlib.verify_request(proof)
            time = elapsed()
            df1 = pd.concat([df1, pd.DataFrame({df.index[i]:{ "verified_value": result, "valid": valid, "verification_seconds": time}}).T])
        
            #print(f"Experiment {i} took {time} seconds and yeilded a result of {result} with a proof size of {proof_size}")
    return pd.concat([df.copy(), df1], axis=1)

In [8]:
folder = "results/debug/"

In [9]:
df_final = final_step_footprinting(df, input_data)
df_final.to_csv(f"{folder}final_step_footprinting.csv", sep=";")

100%|██████████| 100/100 [00:06<00:00, 14.79it/s]


In [10]:
df_unchained = activity_footprinting_unchained(df, input_data)
df_unchained.to_csv(f"{folder}activity_footprinting_unchained.csv", sep=";")

100%|██████████| 100/100 [00:06<00:00, 14.37it/s]


In [11]:
df_composition_unchained = composition_footprinting_unchained(df, df_unchained)
df_composition_unchained.to_csv(f"{folder}composition_footprinting_unchained.csv", sep=";")

100%|██████████| 100/100 [05:41<00:00,  3.41s/it]


In [12]:
df_verified = proof_verification(df_composition_unchained)
df_verified.to_csv(f"{folder}proof_verification.csv", sep=";")

100%|██████████| 100/100 [00:00<00:00, 360.69it/s]


In [13]:
df_chained = activity_footprinting_chained(df, input_data)
df_chained.to_csv(f"{folder}activity_footprinting_chained.csv", sep=";")

100%|██████████| 100/100 [00:07<00:00, 13.49it/s]
